# Excute-argparse-python-script-on-jupyter-notebook

## Chainer test

- [chainer/examples/mnist/train_mnist.py](https://github.com/chainer/chainer/blob/master/examples/mnist/train_mnist.py)

In [1]:
#!/usr/bin/env python
import argparse

import chainer
import chainer.functions as F
import chainer.links as L
from chainer import training
from chainer.training import extensions


# Network definition
class MLP(chainer.Chain):

    def __init__(self, n_units, n_out):
        super(MLP, self).__init__()
        with self.init_scope():
            # the size of the inputs to each layer will be inferred
            self.l1 = L.Linear(None, n_units)  # n_in -> n_units
            self.l2 = L.Linear(None, n_units)  # n_units -> n_units
            self.l3 = L.Linear(None, n_out)  # n_units -> n_out

    def forward(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)


def main():
    parser = argparse.ArgumentParser(description='Chainer example: MNIST')
    parser.add_argument('--batchsize', '-b', type=int, default=100,
                        help='Number of images in each mini-batch')
    parser.add_argument('--epoch', '-e', type=int, default=20,
                        help='Number of sweeps over the dataset to train')
    parser.add_argument('--frequency', '-f', type=int, default=-1,
                        help='Frequency of taking a snapshot')
    parser.add_argument('--gpu', '-g', type=int, default=-1,
                        help='GPU ID (negative value indicates CPU)')
    parser.add_argument('--out', '-o', default='result',
                        help='Directory to output the result')
    parser.add_argument('--resume', '-r', default='',
                        help='Resume the training from snapshot')
    parser.add_argument('--unit', '-u', type=int, default=1000,
                        help='Number of units')
    parser.add_argument('--noplot', dest='plot', action='store_false',
                        help='Disable PlotReport extension')
    args = parser.parse_args()

    print('GPU: {}'.format(args.gpu))
    print('# unit: {}'.format(args.unit))
    print('# Minibatch-size: {}'.format(args.batchsize))
    print('# epoch: {}'.format(args.epoch))
    print('')

    # Set up a neural network to train
    # Classifier reports softmax cross entropy loss and accuracy at every
    # iteration, which will be used by the PrintReport extension below.
    model = L.Classifier(MLP(args.unit, 10))
    if args.gpu >= 0:
        # Make a specified GPU current
        chainer.backends.cuda.get_device_from_id(args.gpu).use()
        model.to_gpu()  # Copy the model to the GPU

    # Setup an optimizer
    optimizer = chainer.optimizers.Adam()
    optimizer.setup(model)

    # Load the MNIST dataset
    train, test = chainer.datasets.get_mnist()

    train_iter = chainer.iterators.SerialIterator(train, args.batchsize)
    test_iter = chainer.iterators.SerialIterator(test, args.batchsize,
                                                 repeat=False, shuffle=False)

    # Set up a trainer
    updater = training.updaters.StandardUpdater(
        train_iter, optimizer, device=args.gpu)
    trainer = training.Trainer(updater, (args.epoch, 'epoch'), out=args.out)

    # Evaluate the model with the test dataset for each epoch
    trainer.extend(extensions.Evaluator(test_iter, model, device=args.gpu))

    # Dump a computational graph from 'loss' variable at the first iteration
    # The "main" refers to the target link of the "main" optimizer.
    trainer.extend(extensions.dump_graph('main/loss'))

    # Take a snapshot for each specified epoch
    frequency = args.epoch if args.frequency == -1 else max(1, args.frequency)
    trainer.extend(extensions.snapshot(), trigger=(frequency, 'epoch'))

    # Write a log of evaluation statistics for each epoch
    trainer.extend(extensions.LogReport())

    # Save two plot images to the result dir
    if args.plot and extensions.PlotReport.available():
        trainer.extend(
            extensions.PlotReport(['main/loss', 'validation/main/loss'],
                                  'epoch', file_name='loss.png'))
        trainer.extend(
            extensions.PlotReport(
                ['main/accuracy', 'validation/main/accuracy'],
                'epoch', file_name='accuracy.png'))

    # Print selected entries of the log to stdout
    # Here "main" refers to the target link of the "main" optimizer again, and
    # "validation" refers to the default name of the Evaluator extension.
    # Entries other than 'epoch' are reported by the Classifier link, called by
    # either the updater or the evaluator.
    trainer.extend(extensions.PrintReport(
        ['epoch', 'main/loss', 'validation/main/loss',
         'main/accuracy', 'validation/main/accuracy', 'elapsed_time']))

    # Print a progress bar to stdout
    trainer.extend(extensions.ProgressBar())

    if args.resume:
        # Resume from a snapshot
        chainer.serializers.load_npz(args.resume, trainer)

    # Run the training
    trainer.run()


In [2]:
# https://stackoverflow.com/a/30662356

if __name__ == "__main__":
    import sys
    sys.argv = ['-f'] + ["--gpu", "1"]
    main()

GPU: 1
# unit: 1000
# Minibatch-size: 100
# epoch: 20



epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time
     total [..................................................]  0.83%
this epoch [########..........................................] 16.67%
       100 iter, 0 epoch / 20 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
     total [..................................................]  1.67%
this epoch [################..................................] 33.33%
       200 iter, 0 epoch / 20 epochs
    258.25 iters/sec. Estimated time to finish: 0:00:45.692877.
     total [#.................................................]  2.50%
this epoch [#########################.........................] 50.00%
       300 iter, 0 epoch / 20 epochs
    258.08 iters/sec. Estimated time to finish: 0:00:45.334674.
     total [#.................................................]  3.33%
this epoch [#################################.................] 66.67%
       400 iter, 0 epoch / 20 epochs

     total [#############.....................................] 26.67%
this epoch [################..................................] 33.33%
      3200 iter, 5 epoch / 20 epochs
    209.75 iters/sec. Estimated time to finish: 0:00:41.954871.
     total [#############.....................................] 27.50%
this epoch [#########################.........................] 50.00%
      3300 iter, 5 epoch / 20 epochs
    210.97 iters/sec. Estimated time to finish: 0:00:41.238743.
     total [##############....................................] 28.33%
this epoch [#################################.................] 66.67%
      3400 iter, 5 epoch / 20 epochs
    212.12 iters/sec. Estimated time to finish: 0:00:40.542322.
     total [##############....................................] 29.17%
this epoch [#########################################.........] 83.33%
      3500 iter, 5 epoch / 20 epochs
    213.21 iters/sec. Estimated time to finish: 0:00:39.867109.
6           0.023185    0.08

     total [##########################........................] 52.50%
this epoch [#########################.........................] 50.00%
      6300 iter, 10 epoch / 20 epochs
    211.14 iters/sec. Estimated time to finish: 0:00:26.995933.
     total [##########################........................] 53.33%
this epoch [#################################.................] 66.67%
      6400 iter, 10 epoch / 20 epochs
    211.73 iters/sec. Estimated time to finish: 0:00:26.449023.
     total [###########################.......................] 54.17%
this epoch [#########################################.........] 83.33%
      6500 iter, 10 epoch / 20 epochs
     212.3 iters/sec. Estimated time to finish: 0:00:25.906181.
11          0.0148841   0.0848824             0.995548       0.9799                    38.9076       
     total [###########################.......................] 55.00%
this epoch [..................................................]  0.00%
      6600 iter, 11 epoc

     total [#######################################...........] 78.33%
this epoch [#################################.................] 66.67%
      9400 iter, 15 epoch / 20 epochs
    211.89 iters/sec. Estimated time to finish: 0:00:12.270398.
     total [#######################################...........] 79.17%
this epoch [#########################################.........] 83.33%
      9500 iter, 15 epoch / 20 epochs
    212.29 iters/sec. Estimated time to finish: 0:00:11.776456.
16          0.0110841   0.10763               0.996849       0.9823                    53.0476       
     total [########################################..........] 80.00%
this epoch [..................................................]  0.00%
      9600 iter, 16 epoch / 20 epochs
    210.31 iters/sec. Estimated time to finish: 0:00:11.411529.
     total [########################################..........] 80.83%
this epoch [########..........................................] 16.67%
      9700 iter, 16 epoc